In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Full MNIST Results 

## Auxiliary functions

In [2]:
def extract_metrics(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, ram_peak * 1024, (ram_peak-ram_init) * 1024, cpu_u + cpu_k)

def extract_metrics_gpu(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
        if "GPU BEFORE EVALUATION:" in line:
            gpu_u, _= eval(line.split(":")[-1])
        if "Peak mem and init mem:" in line:
            gpu_mem = float(line.split(" ")[-2])
        
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, ram_peak * 1024, (ram_peak-ram_init) * 1024, cpu_u + cpu_k, gpu_u, gpu_mem)

In [3]:
def calculate_metrics(files): 
    mlp_metrics = dict()
    cnn_metrics = dict()
    
    for file in files: 
        arch, method, i = file.split("_") 
        metrics = mlp_metrics if arch == "mlp" else cnn_metrics
        
        if method in metrics: 
            metrics[method].append(extract_metrics(file))
        else: 
            metrics[method] = [extract_metrics(file)] 
    
    return (mlp_metrics, cnn_metrics)

def calculate_metrics_gpu(files): 
    mlp_metrics = dict()
    cnn_metrics = dict()
    
    for file in files: 
        arch, method, i = file.split("_") 
        metrics = mlp_metrics if arch == "mlp" else cnn_metrics
        
        if method in metrics: 
            metrics[method].append(extract_metrics_gpu(file))
        else: 
            metrics[method] = [extract_metrics_gpu(file)] 
    
    return (mlp_metrics, cnn_metrics)

In [4]:
def get_means(metrics): 
    return {method: [sum(x)/len(x) for x in list(zip(*metrics[method]))] for method in metrics}

# 1. Task-IL + GPU

In [5]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-13-56/task_gpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics_gpu(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-13-56/task_gpu_fi


## 1.1. MLP Results

In [6]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage', 
                                 'GPU Usage', 
                                 'GPU Memory Consumption'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),98.656667,98.933333,98.776667,99.090000,98.256667,98.553333
Training time (s),63.433333,60.900000,60.766667,58.166667,44.933333,42.100000
Peak RAM Usage (MB),2313.764323,2313.037760,2311.510417,2310.872396,2313.552083,2313.631510
Training-incurred RAM Usage (MB),2.160156,2.524740,2.578125,2.602865,2.627604,2.412760
CPU Usage,92.160000,89.673333,90.320000,89.010000,73.463333,70.566667
GPU Usage,12.214286,12.928571,8.968254,9.555556,4.000000,6.333333
GPU Memory Consumption,991.000000,991.000000,935.000000,935.000000,941.000000,1017.000000


## 1.2. CNN Results

In [7]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage', 
                                 'GPU Usage', 
                                 'GPU Memory Consumption'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),99.160000,99.426667,99.130000,99.496667,98.863333,98.823333
Training time (s),62.666667,64.200000,61.366667,61.233333,44.333333,48.100000
Peak RAM Usage (MB),2316.183594,2316.953125,2315.890625,2319.304688,2315.639323,2312.497396
Training-incurred RAM Usage (MB),2.516927,2.441406,2.378906,2.436198,3.906250,4.385417
CPU Usage,92.166667,93.680000,90.550000,90.340000,72.630000,76.273333
GPU Usage,16.000000,15.238095,10.523810,11.015873,6.066667,8.833333
GPU Memory Consumption,991.000000,991.000000,955.000000,955.000000,963.000000,1039.666667


# 2. Task-IL + No-GPU

In [8]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-15-21/task_nogpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-15-21/task_nogpu_fi


## 2.1. MLP Results

In [9]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),98.880000,99.116667,98.853333,99.113333,99.020000,98.623333
Training time (s),279.166667,283.833333,190.900000,190.200000,37.166667,43.700000
Peak RAM Usage (MB),392.789062,406.998698,336.725260,335.278646,331.188802,427.089844
Training-incurred RAM Usage (MB),77.079427,90.412760,20.729167,20.975260,17.430990,115.717448
CPU Usage,1666.160000,1697.943333,994.530000,992.533333,103.570000,145.380000


## 2.2. CNN Results

In [10]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),99.403333,99.296667,99.206667,99.333333,98.760000,98.983333
Training time (s),369.666667,365.333333,233.100000,233.533333,47.100000,52.366667
Peak RAM Usage (MB),389.986979,405.031250,345.549479,346.019531,344.005208,446.942708
Training-incurred RAM Usage (MB),71.998698,86.898438,28.005208,27.498698,26.395833,129.235677
CPU Usage,2237.673333,2207.973333,1268.086667,1270.123333,154.710000,191.670000


# 3. Class-IL + GPU

In [11]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-18-09/class_gpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics_gpu(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-18-09/class_gpu_fi


## 3.1. MLP Results

In [12]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage', 
                                 'GPU Usage', 
                                 'GPU Memory Consumption'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),93.950000,93.193333,94.493333,94.523333,94.690000,94.703333
Training time (s),106.600000,111.233333,115.133333,112.866667,89.300000,93.100000
Peak RAM Usage (MB),2314.079427,2315.380208,2310.542969,2310.794271,2311.566406,2310.721354
Training-incurred RAM Usage (MB),1.970052,2.255208,2.067708,2.549479,2.596354,2.157552
CPU Usage,164.520000,168.786667,174.053333,171.253333,145.186667,148.996667
GPU Usage,14.921717,13.820707,9.277778,9.083333,3.937037,5.700000
GPU Memory Consumption,991.000000,991.000000,935.000000,935.000000,947.666667,1066.333333


## 3.2. CNN Results

In [13]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage', 
                                 'GPU Usage', 
                                 'GPU Memory Consumption'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),94.856667,94.640000,92.153333,94.443333,94.420000,96.746667
Training time (s),250.533333,246.800000,230.333333,234.500000,184.600000,184.066667
Peak RAM Usage (MB),2319.647135,2316.078125,2312.295573,2314.848958,2312.951823,2316.699219
Training-incurred RAM Usage (MB),2.075521,2.417969,2.971354,2.604167,3.472656,3.270833
CPU Usage,366.670000,359.266667,343.803333,346.363333,295.696667,295.366667
GPU Usage,16.067654,16.243419,10.950145,10.666667,5.727193,8.694932
GPU Memory Consumption,991.000000,991.000000,955.000000,955.000000,969.000000,1089.000000


# 4. Class-IL + No-GPU

In [14]:
%cd /Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-21-25/class_nogpu_fi/

logs = !ls

mlp_metrics, cnn_metrics = calculate_metrics(logs)
mlp_metrics = get_means(mlp_metrics)
cnn_metrics = get_means(cnn_metrics)

/Users/nikhilchuramani/Desktop/Cambridge/PhDWork/Projects/LatentGenerativeReplay/experiments/mnist/2021-05-02-21-25/class_nogpu_fi


## 4.1. MLP Results

In [15]:
pd.DataFrame(mlp_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),93.826667,93.290000,94.663333,94.633333,94.586667,94.850000
Training time (s),567.700000,566.666667,412.033333,419.833333,75.933333,86.400000
Peak RAM Usage (MB),401.343750,417.878906,349.733073,349.373698,338.415365,516.611979
Training-incurred RAM Usage (MB),84.944010,100.621094,35.761719,36.368490,24.701823,204.865885
CPU Usage,3404.523333,3399.503333,2195.426667,2234.736667,214.773333,288.266667


## 4.2. CNN Results

In [16]:
pd.DataFrame(cnn_metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'Peak RAM Usage (MB)' , 
                                 'Training-incurred RAM Usage (MB)', 
                                 'CPU Usage'])

,gr,grd,lgr,lgrd,lr,nr
Accuracy (%),95.196667,94.973333,91.633333,93.556667,94.803333,96.680000
Training time (s),1518.266667,1494.366667,993.133333,984.166667,189.633333,221.200000
Peak RAM Usage (MB),430.130208,432.079427,390.386719,390.927083,352.390625,528.394531
Training-incurred RAM Usage (MB),109.505208,114.552083,71.895833,73.971354,34.397135,210.085938
CPU Usage,9215.200000,9063.303333,5478.453333,5399.463333,631.273333,814.846667
